# Auto ARIMA para séries do TCC

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

c:\Users\matno\anaconda3\envs\quant\lib\site-packages\statsforecast\core.py:25: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
current_path = os.getcwd()

In [4]:
path_series = current_path+"\\Dados\\Tratados\\"

ena = pd.read_csv(path_series+"ena_limpo.csv")
vazao = pd.read_csv(path_series+"vazao_limpo.csv")
carga = pd.read_csv(path_series+"carga_limpo.csv")

dict_series = dict()
dict_series["ena"] = dict()
dict_series["vazao"] = dict()
dict_series["carga"] = dict()

dict_series["ena"]["values"] = ena.ENA
dict_series["ena"]["dates"] = ena.Data
dict_series["carga"]["values"] = carga.Carga
dict_series["carga"]["dates"] = carga.Data
dict_series["vazao"]["values"] = vazao.Vazao
dict_series["vazao"]["dates"] = vazao.Data

In [34]:
series = ["carga", "ena", "vazao"]

for serie in series:
    print(serie)
    y = dict_series[serie]["values"]
    datas = dict_series[serie]["dates"]

    horizon = 12

    y_train = y.iloc[0:-horizon]
    y_test = y.iloc[-horizon:]

    data_train = datas.iloc[0:-horizon]
    data_test = datas.iloc[-horizon:]

    model = AutoARIMA()

    fitted_model = model.fit(y_train.values)
    print("SARIMA (%d,%d,%d)x(%d,%d,%d)"%(fitted_model.model_["arma"][0],fitted_model.model_["arma"][5],fitted_model.model_["arma"][1],
                                        fitted_model.model_["arma"][2],fitted_model.model_["arma"][6],fitted_model.model_["arma"][3]))

    fit_in_sample = fitted_model.predict_in_sample()["fitted"]
    fit_in_sample = pd.DataFrame(dict(Data=data_train, Values=fit_in_sample))

    y_hat = fitted_model.predict(h=horizon)["mean"]
    y_hat = pd.DataFrame(dict(Data=data_test, Values=y_hat))

    fit_in_sample.to_csv(path_series+serie+"_fit_autoarima.csv")
    y_hat.to_csv(path_series+serie+"_forecast_autoarima.csv")

carga
SARIMA (0,1,3)x(0,0,0)
ena
SARIMA (1,0,2)x(0,0,0)
vazao
SARIMA (0,1,3)x(0,0,0)


In [33]:
serie="ena"
y = dict_series[serie]["values"]
datas = dict_series[serie]["dates"]

horizon = 12

y_train = y.iloc[0:-horizon]
y_test = y.iloc[-horizon:]

data_train = datas.iloc[0:-horizon]
data_test = datas.iloc[-horizon:]

model = AutoARIMA()

fitted_model = model.fit(y_train.values)

fit_in_sample = fitted_model.predict_in_sample()["fitted"]
fit_in_sample = pd.DataFrame(dict(Data=data_train, Values=fit_in_sample))

y_hat = fitted_model.predict(h=horizon)["mean"]
y_hat = pd.DataFrame(dict(Data=data_test, Values=y_hat))

print(fitted_model.model_["arma"])

(1, 2, 0, 0, 1, 0, 0)


(1, 2, 0, 0, 1, 0, 0)

In [26]:
fitted_model.model_

{'coef': {'ar1': 0.911248673371839,
  'ma1': 0.22479095848672861,
  'ma2': 0.17075936815851295},
 'sigma2': 126321520.3576787,
 'var_coef': array([[ 2.17849094e-05, -2.74020722e-05, -3.55171815e-05],
        [-4.64386667e-06,  1.75684665e-05,  6.04367923e-06],
        [-6.01914535e-06,  6.04367923e-06,  2.52768389e-05]]),
 'mask': array([ True,  True,  True]),
 'loglik': -2922.1598626173877,
 'aic': 5852.319725234775,
 'arma': (1, 2, 0, 0, 1, 0, 0),
 'residuals': array([ 13482.10020096,  12693.97561121,  -4869.3570658 , -15499.97470661,
         -3285.91149927,   2696.82956818,    399.14225206,    414.25205621,
         11700.35676099, -10743.938921  ,   9412.6476102 ,  13727.78399275,
         -3785.20602683,   1933.10407631,   -403.18885077,  -4658.21363877,
         -1027.64679932,   1075.77438406,  -1078.40738784,   -793.12904977,
          2299.06431459,   6993.20684612,   2492.08951991,  12418.73479363,
         16333.24506685,   8081.74025175, -17224.01046884,  -9354.52360271,
 